In [2]:
import pandas as pd
df = pd.read_json('/home/tukai/git/pythonstuff/python_projects/nobel_winners/nobel_winners_v2.json')

In [3]:
df.count()

born_in     1095
category    1095
country     1095
link        1095
name        1095
text        1095
year        1095
dtype: int64

In [4]:
df.head()

,born_in,category,country,link,name,text,year
0,,Physiology or Medicine,Argentina,http://en.wikipedia.org/wiki/C%C3%A9sar_Milstein,César Milstein,"César Milstein , Physiology or Medicine, 1984",1984
1,,Peace,Argentina,http://en.wikipedia.org/wiki/Adolfo_P%C3%A9rez...,Adolfo Pérez Esquivel,"Adolfo Pérez Esquivel , Peace, 1980",1980
2,,Chemistry,Argentina,http://en.wikipedia.org/wiki/Luis_Federico_Leloir,Luis Federico Leloir,"Luis Federico Leloir , Chemistry, 1970",1970
3,,Physiology or Medicine,Argentina,http://en.wikipedia.org/wiki/Bernardo_Houssay,Bernardo Houssay,"Bernardo Houssay , Physiology or Medicine, 1947",1947
4,,Peace,Argentina,http://en.wikipedia.org/wiki/Carlos_Saavedra_L...,Carlos Saavedra Lamas,"Carlos Saavedra Lamas , Peace, 1936",1936


In [5]:
dupes_by_name = df[df.duplicated('name')]

In [6]:
dupes_by_name.count()

born_in     71
category    71
country     71
link        71
name        71
text        71
year        71
dtype: int64

In [7]:
df = df.drop_duplicates(['name', 'year'])

In [8]:
df.count()

born_in     1030
category    1030
country     1030
link        1030
name        1030
text        1030
year        1030
dtype: int64

In [9]:
from pymongo import MongoClient
def get_mongo_database(db_name, host='localhost',\
                       port=27017, username=None, password=None):
    """ Get named database from MongoDB with/out authentication """
    # make Mongo connection with/out authentication
    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s/%s'%\
        (username, password, host, db_name)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)
    return conn[db_name]

In [10]:
import pandas as pd
from pymongo import MongoClient
def mongo_to_dataframe(db_name, collection, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
    """ create a dataframe from mongodb collection """
    db = get_mongo_database(db_name, host, port, username, password)
    cursor = db[collection].find(query)
    df = pd.DataFrame(list(cursor))
    if no_id:
        del df['_id']
    return df

In [11]:
def dataframe_to_mongo(df, db_name, collection,\
                        host='localhost', port=27017,\
                        username=None, password=None):
    """ save a dataframe to mongodb collection """
    db = get_mongo_database(db_name, host, port, username,\
                            password)
    records = df.to_dict('records')
    db[collection].insert_many(records)

In [12]:
import sqlalchemy

In [13]:
from sqlalchemy import create_engine
import sqlalchemy
engine = create_engine('sqlite:////home/tukai/data/nobel_prize.db', echo=True)

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Enum

In [14]:
class Winner(Base):
    __tablename__ = 'winners'
    index = Column(Integer, primary_key=True)
    born_in = Column(Integer)
    category = Column(String)
    country = Column(String)
    link = Column(String)
    name = Column(String)
    text = Column(String)
    year = Column(Integer)
    
    def __repr__(self):
        return "<Winner(name='%s', category='%s', year='%s')>"\
            %(self.name, self.category, self.year)

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()
session.execute('''DROP TABLE Winners''')
Base.metadata.create_all(engine)

2018-08-17 16:53:12,195 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-08-17 16:53:12,199 INFO sqlalchemy.engine.base.Engine ()
2018-08-17 16:53:12,201 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-08-17 16:53:12,203 INFO sqlalchemy.engine.base.Engine ()
2018-08-17 16:53:12,206 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-08-17 16:53:12,207 INFO sqlalchemy.engine.base.Engine DROP TABLE Winners
2018-08-17 16:53:12,209 INFO sqlalchemy.engine.base.Engine ()
2018-08-17 16:53:12,228 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("winners")
2018-08-17 16:53:12,230 INFO sqlalchemy.engine.base.Engine ()
2018-08-17 16:53:12,235 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE winners (
	"index" INTEGER NOT NULL, 
	born_in INTEGER, 
	category VARCHAR, 
	country VARCHAR, 
	link VARCHAR, 
	name VARCHAR, 
	text VARCHAR, 
	year INTEGER, 
	PRIMARY KEY ("index")
)


2018-08-17 1

In [15]:
df.to_sql('winners', engine, if_exists='append')
session.commit()

2018-08-17 16:53:13,836 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("winners")
2018-08-17 16:53:13,840 INFO sqlalchemy.engine.base.Engine ()
2018-08-17 16:53:13,845 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-08-17 16:53:13,860 INFO sqlalchemy.engine.base.Engine INSERT INTO winners ("index", born_in, category, country, link, name, text, year) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
2018-08-17 16:53:13,862 INFO sqlalchemy.engine.base.Engine ((0, u'', u'Physiology or Medicine', u'Argentina', u'http://en.wikipedia.org/wiki/C%C3%A9sar_Milstein', u'C\xe9sar Milstein', u'C\xe9sar Milstein , Physiology or Medicine, 1984', 1984), (1, u'', u'Peace', u'Argentina', u'http://en.wikipedia.org/wiki/Adolfo_P%C3%A9rez_Esquivel', u'Adolfo P\xe9rez Esquivel', u'Adolfo P\xe9rez Esquivel , Peace, 1980', 1980), (2, u'', u'Chemistry', u'Argentina', u'http://en.wikipedia.org/wiki/Luis_Federico_Leloir', u'Luis Federico Leloir', u'Luis Federico Leloir , Chemistry, 1970', 1970), (3, u'', u'Physiol

In [16]:
session.query(Winner).count()

2018-08-17 16:53:19,851 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-08-17 16:53:19,854 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT winners."index" AS winners_index, winners.born_in AS winners_born_in, winners.category AS winners_category, winners.country AS winners_country, winners.link AS winners_link, winners.name AS winners_name, winners.text AS winners_text, winners.year AS winners_year 
FROM winners) AS anon_1
2018-08-17 16:53:19,856 INFO sqlalchemy.engine.base.Engine ()


1030